In [1]:
import numpy as np 
import pandas as pd 
import gc
import re
import os
train_file = open('train.ft.txt')
test_file = open('test.ft.txt')
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()
del train_file, test_file
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
                                                       
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
del train_file_lines, test_file_lines
labels_test = train_labels[90000:100000]
texts_test = train_sentences[90000:100000]
labels = train_labels[0:90000]
texts = train_sentences[0:90000]
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 200  # We will cut reviews after 100 words
training_samples = 70000  # We will be training on 2000 samples
validation_samples = 20000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(max_words, 64, input_length=maxlen))
model.add(LSTM(64,
               dropout=0.05,
               recurrent_dropout=0.05))
model.add(Dense(1, activation='sigmoid'))
model.summary()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 111047 unique tokens.
Shape of data tensor: (90000, 200)
Shape of label tensor: (90000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 64)           640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 673,089
Trainable params: 673,089
Non-trainable params: 0
_________________________________________________________________


In [2]:
%%time
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=4,
                    batch_size=128)
model.save_weights('lstm_amazon_test.h5')

Epoch 1/4
70000/70000 [==============================] - 158s 2ms/step - loss: 0.3696 - acc: 0.8411
Epoch 2/4
70000/70000 [==============================] - 156s 2ms/step - loss: 0.2443 - acc: 0.9055
Epoch 3/4
70000/70000 [==============================] - 157s 2ms/step - loss: 0.2137 - acc: 0.9189
Epoch 4/4
70000/70000 [==============================] - 157s 2ms/step - loss: 0.1943 - acc: 0.9258
CPU times: user 27min 51s, sys: 5min 6s, total: 32min 58s
Wall time: 10min 29s


In [3]:
sequences = tokenizer.texts_to_sequences(texts_test)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels_test)

In [4]:
%%time
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 11s 1ms/step
CPU times: user 25.5 s, sys: 4.56 s, total: 30 s
Wall time: 10.6 s


[0.22620091429948808, 0.91]